In [ ]:
for symbol in self._all_symbols['symbol']:
                self._logger.info('###########################################')
                # iterate through all the granularity (frequency) from the list
                myPrep.set_symbol(symbol)
                for index, freq in enumerate(const.MY_FREQUENCY_LIST):
                    self._logger.error(f'{symbol} : start execution with granularity {freq}')
                    myPrep.set_data(self.get_candles(symbol, freq))
                    if myPrep.prepare_data_from_FREQ(freq_to_resample(freq)):
                        myStrategy.set_symbol(symbol)
                        if myStrategy.validate_step1(myPrep.get_data_for_validation(step=1)):
                            # we passed the first validation so now we do the next one from the above frequency
                            myPrep.set_next_validation(self.get_candles(symbol, get_above_frequency(freq)) )
                            if myPrep.prepare_data_from_FREQ(freq_to_resample(get_above_frequency(freq))):
                                if myStrategy.validate_step2(myPrep.get_data_for_validation(step=2)):
                                    if myPrep.prep_data_for_trade():
                                        myTrades.get_contract(symbol)
                                        # self.go_trade_in_thread(prep.get_df_for_trade())
                                        # self.go_trade(prep.get_df_for_trade())
                                        display_chart(myPrep._df_main, symbol, freq)
                                        display_chart(myPrep._df_next, symbol, get_above_frequency(freq))
                                        self._logger.error(f'{symbol} : end of search for frequency')
                                        break    
                                    else:
                                        myPrep.reset()
                                else:
                                    myPrep.reset()
                            else:
                                myPrep.reset() 
                        else:
                            myPrep.reset()
                    else:
                        myPrep.reset()

In [ ]:
from pybitget.exceptions import BitgetAPIException
from pybitget import Client
from utils.frequency_utils import getFreq_in_ms
import utils.enums as const
import time
import pandas as pd

if __name__ == '__main__':

    # API_KEY = 'bg_8fd9f7f4f1f2b9ae5b1661eef47d472f'
    # SECRET_KEY = '017ff9852826cf1a30c46af2fabe7d61da87ea5ff6158831eb832d8550d5e565'
    # API_PASSPHRASE = "522723041675"

    # demo key
    API_KEY = 'bg_2885944ddd8087f58b4cdeb6e04cb3c9'
    SECRET_KEY = '5d9cd94afdd157cf04254276da193616f163ae22a31c8efd70551a861ffbd812'
    API_PASSPHRASE = 'demobot5227mtl'

    try:
        myclient = Client(API_KEY, SECRET_KEY, API_PASSPHRASE, verbose=True)
        #tickers = myclient.mix_get_all_tickers('USDT-FUTURES')
        # tickers = myclient.mix_get_all_symbol_ticker('SUSDT-FUTURES')
        #  pour demo SBTCUSDT
        #  pour prod BTCUSDT
        endTime = int(time.time() * 1000)
        startTime = endTime - const.MIN_CANDLES_FOR_INDICATORS * getFreq_in_ms(const.FREQUENCY_5M)
        candles = myclient.mix_get_candles('BTCUSDT','usdt-futures', const.FREQUENCY_5M,startTime, endTime)
        #candles = myclient.mix_get_candles('SBTCSUSDT','susdt-futures', const.FREQUENCY_5M,startTime, endTime)

        # contrat_config_demo for demo only without symbol
        # contract = myclient.mix_get_contract_config_demo('SUSDT-FUTURES')

        # contrat_config for production with symbol
        contract = myclient.mix_get_contract_config('ETHUSDT','USDT-FUTURES')
        df = pd.DataFrame(contract.get('data'))
        df = df.drop(['baseCoin','quoteCoin','feeRateUpRatio','makerFeeRate','takerFeeRate',
                      'openCostUpRatio','supportMarginCoins','symbolType','','maxSymbolOrderNum',
                      'maxProductOrderNum','maxPositionNum','symbolStatus','offTime','limitOpenTime',
                      'deliveryTime','deliveryStartTime', 'deliveryPeriod','launchTime','fundInterval',
                      'minLever','maxLever','posLimit','maintainTime','openTime'],axis=1)
        print(contract)

        account = myclient.mix_get_account('BTCUSDT','USDT-FUTURES','USDT')
        df = pd.DataFrame(contract.get('data'))
        print(account)
        # bids_asks = myclient.mix_get_merge_depth('BTCUSDT','USDT-FUTURES')
        # print(bids_asks)
    except BitgetAPIException as e:
        print("error:" + e.message)
